In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import math
import os

def arima_lstm_model(p):
    n_days_to_predict = 5  # Number of days to predict
    training_period = 15  # Lookback window
    path = "TSLA.csv"  # Path to stock data

    # Load and preprocess the dataset
    headers = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]
    df = pd.read_csv(path, names=headers, skiprows=1)
    df.replace("null", np.nan, inplace=True)
    df[["Open", "High", "Low", "Close", "Adj Close", "Volume"]] = df[["Open", "High", "Low", "Close", "Adj Close", "Volume"]].astype(float)
    df.set_index("Date", inplace=True)
    data = df.filter(['Close'])
    dataset = data.values.flatten()

    # Split into train/test data
    t = math.ceil(len(dataset) * p)
    train_data = dataset[:t]
    test_data = dataset[t:]

    # Step 1: Fit ARIMA model on the training data
    arima_order = (15, 1, 10)  # (p, d, q) order of ARIMA
    arima_model = ARIMA(train_data, order=arima_order)
    arima_model_fit = arima_model.fit()

    # ARIMA predictions
    arima_pred = arima_model_fit.predict(start=training_period, end=t - 1)
    residuals = train_data[training_period:] - arima_pred

    # Normalize the residuals for LSTM training
    scaler = MinMaxScaler(feature_range=(0, 1))
    residuals_scaled = scaler.fit_transform(residuals.reshape(-1, 1))

    # Prepare LSTM training data
    x_train, y_train = [], []
    for i in range(training_period, len(residuals_scaled)):
        x_train.append(residuals_scaled[i - training_period:i, 0])
        y_train.append(residuals_scaled[i, 0])
    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))

    # Step 2: Train LSTM model
    lstm_model = Sequential()
    lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    lstm_model.add(LSTM(units=50, return_sequences=False))
    lstm_model.add(Dense(units=25))
    lstm_model.add(Dense(units=1))
    lstm_model.compile(optimizer='adam', loss='mean_squared_error')
    lstm_model.fit(x_train, y_train, batch_size=22, epochs=50)

    # Step 3: Make predictions
    lstm_predictions = []
    test_residuals = residuals[-training_period:]
    test_residuals_scaled = scaler.transform(test_residuals.reshape(-1, 1))

    x_test = test_residuals_scaled[-training_period:].reshape(1, training_period, 1)
    for _ in range(n_days_to_predict):
        lstm_pred = lstm_model.predict(x_test)
        lstm_predictions.append(scaler.inverse_transform(lstm_pred)[0, 0])
        x_test = np.append(x_test[:, 1:, :], lstm_pred.reshape(1, 1, 1), axis=1)

    # Combine ARIMA and LSTM predictions
    arima_future = arima_model_fit.forecast(steps=n_days_to_predict)
    combined_predictions = arima_future + np.array(lstm_predictions)

    # Plot the results
    plt.figure(figsize=(14, 7))
    # Plot true training data
    plt.plot(data.index[t-20:t], dataset[t-20:t], label='True Training Data', color='blue')

    # Plot true test data (original values)
    plt.plot(data.index[t:t+n_days_to_predict], test_data[:n_days_to_predict], label='True Test Data', color='red')

    # Plot ARIMA predictions
    plt.plot(data.index[t:t+n_days_to_predict], arima_future, label='ARIMA Predictions', color='green')

    # Plot combined predictions
    plt.plot(data.index[t:t+n_days_to_predict], combined_predictions, label='ARIMA-LSTM Predictions', color='orange')

    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title('ARIMA-LSTM Stock Price Prediction')
    plt.legend()
    os.makedirs("TSLA_pred/lstm-arima/15_5", exist_ok=True)
    plt.savefig(f"TSLA_pred/lstm-arima/15_5/arima_lstm_{p:.2f}.png")
    plt.close()

# Run the ARIMA-LSTM model for multiple percentages
for p in np.linspace(0.60, 0.9, 15):
    arima_lstm_model(p)


Epoch 1/50


/home/mili/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0648
Epoch 2/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0085
Epoch 3/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0092
Epoch 4/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0087 
Epoch 5/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0091
Epoch 6/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0081
Epoch 7/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0099
Epoch 8/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0079
Epoch 9/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0086
Epoch 10/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0091
Epoch 11/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0088
Epoch 12/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0085
Epoch 13/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0084
Epoch 14/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0087
Epoch 15/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0082
Epoch 16/50
68/68 ━━

/home/mili/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.0379
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0090
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0098
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0094
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0098
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0101
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0096
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0083
Epoch 9/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0096
Epoch 10/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0098
Epoch 11/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0089
Epoch 12/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0096
Epoch 13/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0101
Epoch 14/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0089
Epoch 15/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0101
Epoch 16/50
70/

...

/home/mili/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


93/93 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0528
Epoch 2/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0018
Epoch 3/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0021
Epoch 4/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0018
Epoch 5/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0021
Epoch 6/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0020
Epoch 7/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0016
Epoch 8/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0015
Epoch 9/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0015
Epoch 10/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 0.0016
Epoch 11/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.0013
Epoch 12/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - loss: 0.0013
Epoch 13/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 0.0015
Epoch 14/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0016
Epoch 15/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 0.0017
Epoch 16/50
93/

/home/mili/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


95/95 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - loss: 0.0490
Epoch 2/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - loss: 0.0016
Epoch 3/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.0017
Epoch 4/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 0.0017
Epoch 5/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0020
Epoch 6/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0019
Epoch 7/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 0.0018
Epoch 8/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.0019
Epoch 9/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.0015
Epoch 10/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0018
Epoch 11/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0017
Epoch 12/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0017
Epoch 13/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 0.0017
Epoch 14/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 0.0018
Epoch 15/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0018
Epoch 16/50
95/

KeyboardInterrupt: 